In [4]:
#For Colab

from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/CV_3
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/CV_3
CV_3_train.ipynb  files		   metrics.py  __pycache__	 templates
data.py		  Flask_Api.ipynb  model.py    README.md	 test_images
dataset		  kaggle.json	   new_data    requirements.txt  train.py
eval.py		  logs		   predict.py  static


In [5]:
!pip install -r requirements.txt

In [7]:
import flask
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import os
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from metrics import dice_loss, dice_coef, iou, dice_loss_m, dice_coef_m
import cv2
import numpy as np

In [6]:
# put your token
!ngrok authtoken 26LPZYNETb2G2mIzWhe9ddHXkYv_6uvhfDYAfqsztiSy9JkZQ

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
 # DeepLabV3 model opening
 with CustomObjectScope({'iou': iou, 'dice_coef_m': dice_coef_m, 'dice_loss_m': dice_loss_m}):
        model = tf.keras.models.load_model("files/model_smooth=0.5.h5")

In [10]:
# Defining prediction and saving function for app
def flask_predict(path):
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  h, w, _ = image.shape
  x = cv2.resize(image, (512, 512))
  x = x/255.0
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis=0)

  """ Prediction """
  y = model.predict(x)[0]
  y = cv2.resize(y, (w, h))
  y = np.expand_dims(y, axis=-1)

  """ Save the image """
  masked_image = image * y
  line = np.ones((h, 10, 3)) * 128
  cat_images = np.concatenate([image, line, masked_image], axis=1)
  cv2.imwrite(f"static/images/result.png", cat_images)

In [11]:
#Defining app
image_folder = os.path.join('static','images/')
app = Flask(__name__)
run_with_ngrok(app) 
app.config["UPLOAD_FOLDER"] = image_folder

@app.route('/', methods=['GET'])
def home():
  return render_template('index.html')

@app.route('/', methods=['POST'])
def predict():

  imagefile = request.files['imagefile']
  image_path = image_folder + imagefile.filename

  # image will be saved in static/images/
  imagefile.save(image_path)

  pic = os.path.join('static/images', imagefile.filename)
  flask_predict(pic)
  return render_template('index.html', user_image="static/images/result.png", prediction_text='Result image and mask')

In [12]:
# Run app and click on http://****.ngrok.io 
if __name__=='__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://682b-34-125-215-76.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Apr/2022 11:05:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 11:05:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Apr/2022 11:05:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2022 11:05:58] "GET /static/images/result.png HTTP/1.1" 200 -
